In [1]:
import pandas as pd
import numpy as np
import qgrid

In [2]:
df = pd.read_csv('./Datasets/NYPD_Complaint_Data_Historic.csv')

<ipython-input-2-63ade1bd11fc>:1: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Datasets/NYPD_Complaint_Data_Historic.csv')


In [3]:
# LAW_CAT_CD is the final output, the rest are input features
data_of_interest = ['CMPLNT_FR_DT','CMPLNT_FR_TM','Latitude','Longitude','VIC_AGE_GROUP','VIC_RACE','VIC_SEX', 'LAW_CAT_CD']
def clean_data(df):
    ret_df = df[data_of_interest].dropna()
    # keep the row which sex is F or M
    ret_df = ret_df.loc[ret_df['VIC_SEX'].isin(['F','M'])]
    # keep the row with known age group
    ret_df = ret_df.loc[ret_df['VIC_AGE_GROUP'] != 'UNKNOWN']
    ret_df = ret_df.loc[ret_df['VIC_RACE'] != 'UNKNOWN']
    
    # remove the year from date
    ret_df['CMPLNT_FR_DT'] = ret_df['CMPLNT_FR_DT'].apply(lambda x: x.rsplit('/',1)[0])
    ret_df['Timestamp'] = ret_df['CMPLNT_FR_DT'] + ' ' + ret_df['CMPLNT_FR_TM']
    ret_df = ret_df.drop(['CMPLNT_FR_DT','CMPLNT_FR_TM'], axis=1)
    ret_df = ret_df[['Timestamp','Latitude','Longitude','VIC_AGE_GROUP','VIC_RACE','VIC_SEX', 'LAW_CAT_CD']]
    return ret_df

In [4]:
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,394506329,12/31/2019,17:30:00,NaN,NaN,32.0,12/31/2019,118,DANGEROUS WEAPONS,793.0,...,NaN,NaN,40.820927,-73.943324,"(40.82092679700002, -73.94332421899996)",PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,E
1,968873685,12/29/2019,16:31:00,12/29/2019,16:54:00,47.0,12/29/2019,113,FORGERY,729.0,...,NaN,NaN,40.885701,-73.861640,"(40.885701406000074, -73.86164032499995)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E
2,509837549,12/15/2019,18:45:00,NaN,NaN,109.0,12/29/2019,578,HARRASSMENT 2,638.0,...,M,NaN,40.742281,-73.819824,"(40.74228115600005, -73.81982408)",PATROL BORO QUEENS NORTH,NaN,25-44,WHITE HISPANIC,F
3,352454313,12/28/2019,01:00:00,NaN,NaN,47.0,12/28/2019,126,MISCELLANEOUS PENAL LAW,117.0,...,M,NaN,40.875311,-73.847545,"(40.87531145100007, -73.84754521099995)",PATROL BORO BRONX,NaN,UNKNOWN,UNKNOWN,E
4,248803469,09/05/2008,21:41:00,NaN,NaN,NaN,09/05/2008,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,NaN,40.698827,-73.938819,"(40.698827283, -73.938819047)",NaN,NaN,25-44,BLACK,M


In [5]:
len(df['OFNS_DESC'].unique())

72

In [7]:
len(df['PD_CD'].unique())

433

In [12]:
df_interest = clean_data(df.iloc[:100])

In [13]:
df_interest

,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,LAW_CAT_CD
0,12/31/2019,17:30:00,40.820927,-73.943324,UNKNOWN,UNKNOWN,E,FELONY
1,12/29/2019,16:31:00,40.885701,-73.861640,UNKNOWN,UNKNOWN,E,FELONY
2,12/15/2019,18:45:00,40.742281,-73.819824,25-44,WHITE HISPANIC,F,VIOLATION
3,12/28/2019,01:00:00,40.875311,-73.847545,UNKNOWN,UNKNOWN,E,FELONY
4,09/05/2008,21:41:00,40.698827,-73.938819,25-44,BLACK,M,FELONY
...,...,...,...,...,...,...,...,...
95,11/25/2019,15:00:00,40.766002,-73.822316,65+,BLACK,M,MISDEMEANOR
96,07/01/2019,00:00:00,40.823101,-73.869690,<18,WHITE HISPANIC,F,FELONY
97,11/25/2019,16:35:00,40.574286,-74.105914,UNKNOWN,UNKNOWN,E,MISDEMEANOR
98,11/24/2019,20:00:00,40.739216,-73.884567,25-44,BLACK,M,VIOLATION


In [15]:
df_interest['VIC_SEX'].unique()

array(['E', 'F', 'M', 'D'], dtype=object)

In [17]:
df_sex = df_interest.loc[df_interest['VIC_SEX'].isin(['F','M'])]
df_sex

,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,LAW_CAT_CD
2,12/15/2019,18:45:00,40.742281,-73.819824,25-44,WHITE HISPANIC,F,VIOLATION
4,09/05/2008,21:41:00,40.698827,-73.938819,25-44,BLACK,M,FELONY
5,12/27/2019,22:00:00,40.720759,-73.980466,UNKNOWN,UNKNOWN,M,FELONY
7,12/26/2019,20:00:00,40.611570,-73.747365,25-44,WHITE HISPANIC,F,MISDEMEANOR
8,12/26/2019,19:57:00,40.840234,-73.917684,18-24,BLACK,F,MISDEMEANOR
...,...,...,...,...,...,...,...,...
94,01/21/2007,10:23:00,NaN,NaN,25-44,BLACK,M,FELONY
95,11/25/2019,15:00:00,40.766002,-73.822316,65+,BLACK,M,MISDEMEANOR
96,07/01/2019,00:00:00,40.823101,-73.869690,<18,WHITE HISPANIC,F,FELONY
98,11/24/2019,20:00:00,40.739216,-73.884567,25-44,BLACK,M,VIOLATION


In [19]:
df_age = df_sex[df_sex['VIC_AGE_GROUP'] != 'UNKNOWN']
df_age

,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,LAW_CAT_CD
2,12/15/2019,18:45:00,40.742281,-73.819824,25-44,WHITE HISPANIC,F,VIOLATION
4,09/05/2008,21:41:00,40.698827,-73.938819,25-44,BLACK,M,FELONY
7,12/26/2019,20:00:00,40.611570,-73.747365,25-44,WHITE HISPANIC,F,MISDEMEANOR
8,12/26/2019,19:57:00,40.840234,-73.917684,18-24,BLACK,F,MISDEMEANOR
9,12/25/2019,23:00:00,40.772875,-73.989542,45-64,BLACK,M,MISDEMEANOR
...,...,...,...,...,...,...,...,...
94,01/21/2007,10:23:00,NaN,NaN,25-44,BLACK,M,FELONY
95,11/25/2019,15:00:00,40.766002,-73.822316,65+,BLACK,M,MISDEMEANOR
96,07/01/2019,00:00:00,40.823101,-73.869690,<18,WHITE HISPANIC,F,FELONY
98,11/24/2019,20:00:00,40.739216,-73.884567,25-44,BLACK,M,VIOLATION


In [20]:
df_race = df_age[df_age['VIC_RACE'] != 'UNKNOWN']
df_race

,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,LAW_CAT_CD
2,12/15/2019,18:45:00,40.742281,-73.819824,25-44,WHITE HISPANIC,F,VIOLATION
4,09/05/2008,21:41:00,40.698827,-73.938819,25-44,BLACK,M,FELONY
7,12/26/2019,20:00:00,40.611570,-73.747365,25-44,WHITE HISPANIC,F,MISDEMEANOR
8,12/26/2019,19:57:00,40.840234,-73.917684,18-24,BLACK,F,MISDEMEANOR
9,12/25/2019,23:00:00,40.772875,-73.989542,45-64,BLACK,M,MISDEMEANOR
...,...,...,...,...,...,...,...,...
93,11/21/2019,20:00:00,40.714248,-73.807354,45-64,ASIAN / PACIFIC ISLANDER,F,VIOLATION
94,01/21/2007,10:23:00,NaN,NaN,25-44,BLACK,M,FELONY
95,11/25/2019,15:00:00,40.766002,-73.822316,65+,BLACK,M,MISDEMEANOR
96,07/01/2019,00:00:00,40.823101,-73.869690,<18,WHITE HISPANIC,F,FELONY


In [23]:
df_nona = df_race.dropna()

In [24]:
df_nona

,CMPLNT_FR_DT,CMPLNT_FR_TM,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,LAW_CAT_CD
2,12/15/2019,18:45:00,40.742281,-73.819824,25-44,WHITE HISPANIC,F,VIOLATION
4,09/05/2008,21:41:00,40.698827,-73.938819,25-44,BLACK,M,FELONY
7,12/26/2019,20:00:00,40.611570,-73.747365,25-44,WHITE HISPANIC,F,MISDEMEANOR
8,12/26/2019,19:57:00,40.840234,-73.917684,18-24,BLACK,F,MISDEMEANOR
9,12/25/2019,23:00:00,40.772875,-73.989542,45-64,BLACK,M,MISDEMEANOR
...,...,...,...,...,...,...,...,...
92,11/26/2019,23:30:00,40.887451,-73.847608,<18,BLACK,F,FELONY
93,11/21/2019,20:00:00,40.714248,-73.807354,45-64,ASIAN / PACIFIC ISLANDER,F,VIOLATION
95,11/25/2019,15:00:00,40.766002,-73.822316,65+,BLACK,M,MISDEMEANOR
96,07/01/2019,00:00:00,40.823101,-73.869690,<18,WHITE HISPANIC,F,FELONY


In [110]:
df1 = clean_data(df.iloc[:100])
df1

,Timestamp,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,LAW_CAT_CD
2,12/15 18:45:00,40.742281,-73.819824,25-44,WHITE HISPANIC,F,VIOLATION
4,09/05 21:41:00,40.698827,-73.938819,25-44,BLACK,M,FELONY
7,12/26 20:00:00,40.611570,-73.747365,25-44,WHITE HISPANIC,F,MISDEMEANOR
8,12/26 19:57:00,40.840234,-73.917684,18-24,BLACK,F,MISDEMEANOR
9,12/25 23:00:00,40.772875,-73.989542,45-64,BLACK,M,MISDEMEANOR
...,...,...,...,...,...,...,...
92,11/26 23:30:00,40.887451,-73.847608,<18,BLACK,F,FELONY
93,11/21 20:00:00,40.714248,-73.807354,45-64,ASIAN / PACIFIC ISLANDER,F,VIOLATION
95,11/25 15:00:00,40.766002,-73.822316,65+,BLACK,M,MISDEMEANOR
96,07/01 00:00:00,40.823101,-73.869690,<18,WHITE HISPANIC,F,FELONY


In [5]:
df_manhattan = df[df['BORO_NM']=='MANHATTAN']

In [7]:
df_manhattan.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,394506329,12/31/2019,17:30:00,NaN,NaN,32.0,12/31/2019,118,DANGEROUS WEAPONS,793.0,...,NaN,NaN,40.820927,-73.943324,"(40.82092679700002, -73.94332421899996)",PATROL BORO MAN NORTH,NaN,UNKNOWN,UNKNOWN,E
5,293718737,12/27/2019,22:00:00,NaN,NaN,9.0,12/27/2019,107,BURGLARY,223.0,...,NaN,NaN,40.720759,-73.980466,"(40.72075882100006, -73.98046642299995)",PATROL BORO MAN SOUTH,NaN,UNKNOWN,UNKNOWN,M
9,241602326,12/25/2019,23:00:00,12/26/2019,14:25:00,20.0,12/26/2019,341,PETIT LARCENY,321.0,...,NaN,NaN,40.772875,-73.989542,"(40.77287456000005, -73.98954212299998)",PATROL BORO MAN NORTH,NaN,45-64,BLACK,M
10,585574881,12/24/2019,16:00:00,NaN,NaN,14.0,12/24/2019,109,GRAND LARCENY,433.0,...,NaN,NaN,40.750431,-73.989282,"(40.75043076800005, -73.98928217599996)",PATROL BORO MAN SOUTH,NaN,UNKNOWN,UNKNOWN,D
11,916172359,12/21/2019,20:00:00,12/21/2019,20:30:00,25.0,12/24/2019,341,PETIT LARCENY,301.0,...,U,NaN,40.801684,-73.938959,"(40.80168405900002, -73.93895929199994)",PATROL BORO MAN NORTH,NaN,25-44,UNKNOWN,F


In [8]:
df_clean = clean_data(df_manhattan)
len(df_clean)

1085623

In [9]:
df_clean.head(10)

,Timestamp,Latitude,Longitude,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,LAW_CAT_CD
9,12/25 23:00:00,40.772875,-73.989542,45-64,BLACK,M,MISDEMEANOR
29,12/15 04:15:00,40.807781,-73.945485,25-44,WHITE,F,VIOLATION
34,12/05 22:00:00,40.828851,-73.943834,25-44,WHITE,F,MISDEMEANOR
47,12/10 15:30:00,40.816509,-73.946547,<18,BLACK,F,MISDEMEANOR
54,12/08 23:20:00,40.789633,-73.940068,25-44,WHITE,M,VIOLATION
67,04/01 00:00:00,40.784060,-73.975021,45-64,WHITE,F,FELONY
71,11/30 14:00:00,40.853725,-73.926802,18-24,WHITE HISPANIC,M,FELONY
84,11/29 21:50:00,40.754517,-73.984358,25-44,WHITE,F,MISDEMEANOR
113,11/18 18:30:00,40.743968,-73.977912,25-44,WHITE,F,VIOLATION
117,11/16 18:00:00,40.806082,-73.940404,<18,BLACK,F,MISDEMEANOR


In [10]:
df_clean.to_csv('./Datasets/dataset.csv',index=False, header=['Timestamp','Latitude','Longitude','Victim_AGE','Victim_RACE','Victim_Sex', 'Category'])